In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor
import joblib


ModuleNotFoundError: No module named 'sklearn'

In [2]:
!pip install scikit-learn

  Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl (8.0 MB)
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached scipy-1.16.3-cp313-cp313-win_amd64.whl (38.5 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ------------------

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor
import joblib

In [4]:
df = pd.read_csv("../artifacts/gym_members_exercise_tracking.csv")

In [5]:
df.head(10)

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39
5,56,Female,58.0,1.68,168,156,74,1.59,1116.0,HIIT,15.5,2.7,5,3,20.55
6,36,Male,70.3,1.72,174,169,73,1.49,1385.0,Cardio,21.3,2.3,3,2,23.76
7,40,Female,69.7,1.51,189,141,64,1.27,895.0,Cardio,30.6,1.9,3,2,30.57
8,28,Male,121.7,1.94,185,127,52,1.03,719.0,Strength,28.9,2.6,4,2,32.34
9,28,Male,101.8,1.84,169,136,64,1.08,808.0,Cardio,29.7,2.7,3,1,30.07


In [6]:
def feature_engineering(data):
    data = data.copy()
    
    data["HR_Reserve"] = data["Max_BPM"] - data["Resting_BPM"]
    data["Intensity_Ratio"] = data["Avg_BPM"] / data["Max_BPM"]
    data["Workout_Load"] = data["Avg_BPM"] * data["Session_Duration (hours)"]
    
    return data


In [7]:
df = feature_engineering(df)

In [8]:
df.head(10)

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,HR_Reserve,Intensity_Ratio,Workout_Load
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20,120,0.872222,265.33
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00,113,0.843575,196.30
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71,113,0.730539,135.42
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41,134,0.863158,96.76
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39,120,0.840426,101.12
5,56,Female,58.0,1.68,168,156,74,1.59,1116.0,HIIT,15.5,2.7,5,3,20.55,94,0.928571,248.04
6,36,Male,70.3,1.72,174,169,73,1.49,1385.0,Cardio,21.3,2.3,3,2,23.76,101,0.971264,251.81
7,40,Female,69.7,1.51,189,141,64,1.27,895.0,Cardio,30.6,1.9,3,2,30.57,125,0.746032,179.07
8,28,Male,121.7,1.94,185,127,52,1.03,719.0,Strength,28.9,2.6,4,2,32.34,133,0.686486,130.81
9,28,Male,101.8,1.84,169,136,64,1.08,808.0,Cardio,29.7,2.7,3,1,30.07,105,0.804734,146.88


In [9]:
X = df.drop("Calories_Burned", axis=1)
y = df["Calories_Burned"]


In [10]:
df.head(10)

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,HR_Reserve,Intensity_Ratio,Workout_Load
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20,120,0.872222,265.33
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00,113,0.843575,196.30
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71,113,0.730539,135.42
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41,134,0.863158,96.76
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39,120,0.840426,101.12
5,56,Female,58.0,1.68,168,156,74,1.59,1116.0,HIIT,15.5,2.7,5,3,20.55,94,0.928571,248.04
6,36,Male,70.3,1.72,174,169,73,1.49,1385.0,Cardio,21.3,2.3,3,2,23.76,101,0.971264,251.81
7,40,Female,69.7,1.51,189,141,64,1.27,895.0,Cardio,30.6,1.9,3,2,30.57,125,0.746032,179.07
8,28,Male,121.7,1.94,185,127,52,1.03,719.0,Strength,28.9,2.6,4,2,32.34,133,0.686486,130.81
9,28,Male,101.8,1.84,169,136,64,1.08,808.0,Cardio,29.7,2.7,3,1,30.07,105,0.804734,146.88


In [11]:
X

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,HR_Reserve,Intensity_Ratio,Workout_Load
0,56,Male,88.3,1.71,180,157,60,1.69,Yoga,12.6,3.5,4,3,30.20,120,0.872222,265.33
1,46,Female,74.9,1.53,179,151,66,1.30,HIIT,33.9,2.1,4,2,32.00,113,0.843575,196.30
2,32,Female,68.1,1.66,167,122,54,1.11,Cardio,33.4,2.3,4,2,24.71,113,0.730539,135.42
3,25,Male,53.2,1.70,190,164,56,0.59,Strength,28.8,2.1,3,1,18.41,134,0.863158,96.76
4,38,Male,46.1,1.79,188,158,68,0.64,Strength,29.2,2.8,3,1,14.39,120,0.840426,101.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,24,Male,87.1,1.74,187,158,67,1.57,Strength,10.0,3.5,4,3,28.77,120,0.844920,248.06
969,25,Male,66.6,1.61,184,166,56,1.38,Strength,25.0,3.0,2,1,25.69,128,0.902174,229.08
970,59,Female,60.4,1.76,194,120,53,1.72,Cardio,18.8,2.7,5,3,19.50,141,0.618557,206.40
971,32,Male,126.4,1.83,198,146,62,1.10,HIIT,28.2,2.1,3,2,37.74,136,0.737374,160.60


In [12]:
numeric_features = [
    "Age", "Weight (kg)", "Height (m)", "BMI", "Fat_Percentage",
    "Max_BPM", "Avg_BPM", "Resting_BPM",
    "Session_Duration (hours)", "Water_Intake (liters)",
    "Workout_Frequency (days/week)", "Experience_Level",
    "HR_Reserve", "Intensity_Ratio", "Workout_Load"
]

categorical_features = [
    "Gender", "Workout_Type"
]


In [13]:
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])


In [14]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


In [15]:
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=200,
        random_state=42
    ))
])


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [17]:
X_train

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI,HR_Reserve,Intensity_Ratio,Workout_Load
967,20,Male,55.0,1.60,172,168,67,1.12,Yoga,24.0,3.2,4,2,21.48,105,0.976744,188.16
365,39,Female,60.6,1.65,162,167,63,0.92,HIIT,27.6,1.7,3,1,22.26,99,1.030864,153.64
559,54,Female,75.6,1.72,194,154,60,0.97,Cardio,26.4,1.9,3,1,25.55,134,0.793814,149.38
33,24,Female,58.9,1.51,187,157,68,1.04,Cardio,31.7,2.5,2,1,25.83,119,0.839572,163.28
31,20,Female,65.4,1.52,185,127,50,1.03,Yoga,28.0,2.2,4,2,28.31,135,0.686486,130.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,41,Male,71.7,1.96,162,168,51,0.51,Yoga,27.3,2.5,2,1,18.66,111,1.037037,85.68
270,19,Female,58.8,1.77,181,136,53,1.40,HIIT,32.8,2.7,2,1,18.77,128,0.751381,190.40
860,23,Female,56.7,1.55,179,132,73,1.38,Cardio,34.0,1.9,4,2,23.60,106,0.737430,182.16
435,51,Female,56.3,1.50,182,142,66,1.43,Cardio,27.4,2.4,3,1,25.02,116,0.780220,203.06


In [18]:
X_test.count()

Age                              195
Gender                           195
Weight (kg)                      195
Height (m)                       195
Max_BPM                          195
Avg_BPM                          195
Resting_BPM                      195
Session_Duration (hours)         195
Workout_Type                     195
Fat_Percentage                   195
Water_Intake (liters)            195
Workout_Frequency (days/week)    195
Experience_Level                 195
BMI                              195
HR_Reserve                       195
Intensity_Ratio                  195
Workout_Load                     195
dtype: int64

In [19]:
model_pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [20]:
y_pred = model_pipeline.predict(X_test)


In [21]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("MAE :", mae)
print("RMSE:", rmse)
print("R2  :", r2)


MAE : 12.039641025641023
RMSE: 22.70699054792576
R2  : 0.9938195114022232


In [22]:
sample_input = pd.DataFrame([{
    "Age": 25,
    "Gender": "Male",
    "Weight (kg)": 70,
    "Height (m)": 1.75,
    "BMI": 22.9,
    "Fat_Percentage": 18,
    "Max_BPM": 190,
    "Avg_BPM": 150,
    "Resting_BPM": 65,
    "Session_Duration (hours)": 1.2,
    "Water_Intake (liters)": 2.0,
    "Workout_Frequency (days/week)": 4,
    "Experience_Level": 2,
    "Workout_Type": "Cardio"
}])

sample_input = feature_engineering(sample_input)
prediction = model_pipeline.predict(sample_input)

print("Predicted Calories Burned:", prediction[0])


Predicted Calories Burned: 991.595


In [23]:
sample_input = pd.DataFrame([{
    "Age": 25,
    "Gender": "Male",
    "Weight (kg)": 70,
    "Height (m)": 1.75,
    "BMI": 22.9,
    "Fat_Percentage": 18,
    "Max_BPM": ,
    "Avg_BPM": 150,
    "Resting_BPM": 65,
    "Session_Duration (hours)": 1.2,
    "Water_Intake (liters)": 2.0,
    "Workout_Frequency (days/week)": 4,
    "Experience_Level": 2,
    "Workout_Type": "Cardio"
}])

sample_input = feature_engineering(sample_input)
prediction = model_pipeline.predict(sample_input)

print("Predicted Calories Burned:", prediction[0])


SyntaxError: expression expected after dictionary key and ':' (1484442798.py, line 8)

In [24]:
joblib.dump(model_pipeline, "..\model\calories_burned_pipeline.pkl")

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\yasin\AppData\Local\Temp\ipykernel_19232\402469335.py:1: SyntaxWarning: invalid escape sequence '\m'
  joblib.dump(model_pipeline, "..\model\calories_burned_pipeline.pkl")


['..\\model\\calories_burned_pipeline.pkl']

In [25]:
sample_input = pd.DataFrame([{
    "Age": 25,
    "Gender": "Male",
    "Weight (kg)": 70,
    "Height (m)": 1.75,
    "BMI": 22.9,
    "Fat_Percentage": 18,
    "Max_BPM": 190,
    "Avg_BPM": 120,
    "Resting_BPM": 65,
    "Session_Duration (hours)": 1.2,
    "Water_Intake (liters)": 2.0,
    "Workout_Frequency (days/week)": 4,
    "Experience_Level": 2,
    "Workout_Type": "Cardio"
}])

sample_input = feature_engineering(sample_input)
prediction = model_pipeline.predict(sample_input)

print("Predicted Calories Burned:", prediction[0])

Predicted Calories Burned: 787.565
